In [1]:
user = "Alex"
pathtorepo = "C:\\Users\\" *user *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\bifurcation\\")

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\bifurcation`


In [2]:
using BifurcationKit, HclinicBifurcationKit, Setfield, LinearAlgebra, Plots, Parameters

In [3]:
τ_ = 0.013; τD_ = 0.07993;  τy_ = 3.3; J_ = 3.07; β_ = 0.300
xthr_ = 0.75; ythr_ = 0.4   
α_ = 1.58; ΔU0_ = 0.305
I0_ = -1.7064; U0_ = 0.265; 

In [4]:
@inbounds function TM_bk(u, p)
    U(y, p) = p[8] + p[9] / ( 1.0 + exp( -50.0 * (y - p[7]) ) )
    σ(x, p) = 1.0 / ( 1.0 + exp( -20.0 * (x-p[6]) ) )
    g(E, x, y, p, U_) = log( 1.0 + exp( (p[5] * U_ * x * E + p[11]  ) / (p[1]) ) )
    
    U_ = U(u[3], p)
    du1 = (-u[1] + p[1] * g(u[1], u[2], u[3], p, U_) ) / p[2]
    du2 = (1.0 - u[2]) / p[3] - U_*u[2]*u[1]
    du3 = (-u[3])/p[4] + p[10] * σ(u[2], p)
    
    return [du1, du2, du3]
end

TM_bk (generic function with 1 method)

In [5]:
p = (α = α_, τ = τ_, τD = τD_, τy = τy_, J = J_, xthr = xthr_, ythr = ythr_, U0 = U0_, ΔU0 = ΔU0_, β = β_, I0 = I0_)

(α = 1.58, τ = 0.013, τD = 0.07993, τy = 3.3, J = 3.07, xthr = 0.75, ythr = 0.4, U0 = 0.265, ΔU0 = 0.305, β = 0.3, I0 = -1.7064)

In [6]:
fp0 = [ 8.34581,  0.738495,  0.438299];

In [7]:
prob =  BifurcationProblem(TM_bk, fp0, p, (@lens _.I0))

┌─ Bifurcation Problem with uType Vector{Float64}
├─ Inplace:  false
├─ Symmetric: false
└─ Parameter: I0

In [8]:
opt_new = NewtonPar(maxIter = 50, tol = 1e-6) # maxIter = 3
pmax, pmin = 0.0, -1.74

(0.0, -1.74)

In [9]:
opts_con = ContinuationPar(pMin = pmin, pMax = pmax,
                            ds = 0.001, dsmin = 1e-5, dsmax = 0.1,
                            nev = 3, detectBifurcation = 3, newtonOptions  = opt_new,
                            maxSteps  = 300)

ContinuationPar{Float64, DefaultLS, DefaultEig{typeof(real)}}
  dsmin: Float64 1.0e-5
  dsmax: Float64 0.1
  ds: Float64 0.001
  a: Float64 0.5
  pMin: Float64 -1.74
  pMax: Float64 0.0
  maxSteps: Int64 300
  newtonOptions: NewtonPar{Float64, DefaultLS, DefaultEig{typeof(real)}}
  η: Float64 150.0
  saveToFile: Bool false
  saveSolEveryStep: Int64 1
  nev: Int64 3
  saveEigEveryStep: Int64 1
  saveEigenvectors: Bool true
  plotEveryStep: Int64 10
  tolStability: Float64 1.0e-10
  detectFold: Bool true
  detectBifurcation: Int64 3
  dsminBisection: Float64 1.0e-16
  nInversion: Int64 2
  maxBisectionSteps: Int64 15
  tolBisectionEigenvalue: Float64 1.0e-16
  detectEvent: Int64 0
  tolParamBisectionEvent: Float64 1.0e-16
  detectLoop: Bool false


In [10]:
opt_new

NewtonPar{Float64, DefaultLS, DefaultEig{typeof(real)}}
  tol: Float64 1.0e-6
  maxIter: Int64 50
  verbose: Bool false
  linsolver: DefaultLS
  eigsolver: DefaultEig{typeof(real)}
  linesearch: Bool false
  α: Float64 1.0
  αmin: Float64 0.001


In [11]:
br = continuation(prob, PALC(), opts_con)

In [ ]:
plot(br)

In [ ]:
hp_codim2_1 = continuation(br, 1, (@lens _.U0),
	ContinuationPar(opts_con, pMin = 0.0, pMax = 0.7,
		ds = 0.001, dsmax = 0.1, dsmin = 1e-5),
	# detection of codim 2 bifurcations with bisection
	detectCodim2Bifurcation = 2,
	# tell to start the Hopf problem using eigen elements: compute left eigenvector
	startWithEigen = true,
	# we update the Hopf problem at every continuation step
	updateMinAugEveryStep = 1,
	# compute both sides of the initial condition
	bothside = true,
    verbosity = 3
	)

In [ ]:
hp_codim2_1

In [ ]:
plot(hp_codim2_1)

In [ ]:
solbt = newton(hp_codim2_1, 2; options = NewtonPar(hp_codim2_1.contparams.newtonOptions, verbose = true));

@set! hp_codim2_1.specialpoint[2].param = solbt.u.params.U0
@set! hp_codim2_1.specialpoint[2].printsol.I0 = solbt.u.params.I0
@set! hp_codim2_1.specialpoint[2].printsol.U0 = solbt.u.params.U0
hp_codim2_1.specialpoint[2].x[1:3] .= solbt.u.x0

# predictors from the BT normal form. These are by no means accurate!
Hom = BK.predictor(btpt, Val(:HomoclinicCurve), 0.01)
_S = LinRange(-0.25, 0.55, 1000)
plot(hp_codim2_1)
plot!([Hom.α(s)[2] for s in _S], [Hom.α(s)[1] for s in _S], linewidth=5, label = "Hom curve predictor")

# plot of the homoclinic orbit
hom1 = [Hom.orbit(t,0.1)[1] for t in LinRange(-1000, 1000, 10000)]
hom2 = [Hom.orbit(t,0.1)[2] for t in LinRange(-1000, 1000, 10000)]
plot(hom1, hom2, title = "predictor for Hom. solution", label = "")

In [ ]:
tpt = getNormalForm(hp_codim2_1, 2; nev = 3, autodiff = false)

using HclinicBifurcationKit

function plotHom(x,p;k...)
    𝐇𝐨𝐦 = p.prob
    par0 = set(BK.getParams(𝐇𝐨𝐦), BK.getLens(𝐇𝐨𝐦), x.x[end][1])
    par0 = set(par0, p.lens, p.p)
    sol = getHomoclinicOrbit(𝐇𝐨𝐦, x, par0)
    m = (𝐇𝐨𝐦.bvp isa PeriodicOrbitOCollProblem && 𝐇𝐨𝐦.bvp.meshadapt) ? :d : :none
    plot!(sol.t, sol[1,:],subplot=3, markersize = 1, marker=m)
    # plot!(sol[1,:], sol[2,:],subplot=3, markersize = 1, marker=m)
end

# using DifferentialEquations
# probsh = ODEProblem(TM_bk!, copy(btpt.x0), (0., 100.), p_tm; abstol = 1e-12, reltol = 1e-10)
# sol = solve(probsh, Rodas5())
# plot(sol, vars=(:t,1))

opts_con_hom = @set opts_con.newtonOptions.verbose = true
@set! opts_con_hom.newtonOptions.maxIter = 15
br_hom_c = continuation(
        prob,
        btpt,
        # we use mesh adaptation
        PeriodicOrbitOCollProblem(90, 4; meshadapt = true, K = 500),
        PALC(tangent = Bordered()),
        setproperties(opts_con_hom, maxSteps = 130, saveSolEveryStep = 1, dsmax = 3e-2, plotEveryStep = 1, pMin = -1.01, pMax = 0.4, ds = 0.0001, detectEvent = 2, detectBifurcation = 0);
        verbosity = 1, plot = true,
        ϵ0 = 1e-8, amplitude = 1e-3,
        freeparams = ((@lens _.ϵ0), (@lens _.ϵ1)),
        plotSolution = plotHom,
        callbackN = BK.cbMaxNorm(1e0),
        )

# plot the first homoclinic solution        
solh = HclinicBifurcationKit.getHomoclinicOrbit(br_hom_c, 1)
plot(solh.t, solh[1,:])

plot(hp_codim2_1, sn_codim2_1, branchlabel = ["Fold", "Hopf"])
plot!(br_hom_c, branchlabel = "Hom")